## Importing modules and packages

In [4]:
# Importing packages
import osmnx  as ox
import pandas as pd
import numpy  as np
import folium
import os.path
import shapely
import geopandas as gpd
from IPython.display import IFrame
from csv import writer, reader

# Importing modules
import gr_mapmatch # Contains functions that perform the map matching of roads
import gr_placematch # Contains functions that perform the map matching of places
import gr_utils # Contains useful geometry functions
import gr_plot # Contains plotting routines
import gr_process

# Configuring modules & packages
ox.settings.useful_tags_way = [
    "bridge","tunnel","name","highway","area","landuse","surface","tracktype"
] # Configuring which parameters we want to obtain from OSM

## Input parameters

In [5]:
trailname = 'gr122' # Name of the hiking trail to be considered (will search for trail.csv or trail.gpx as sources)

# Settings for trail2roads
points_per_batch = 100 # Subdivide the trail into batches of this many points
delta = 0.0100 # Tolerance around bounding box per trail section [deg]

# Settings for roads2places
points_per_batch_places = 100 # Subdivide the trail into batches of this many segments
delta_places = 0.015 # bbox delta in deg
buffersize = 0.00015 # buffer used when merging landuse places
tol_area = 15.0e-6 # buffer used to define which merged landuse places we consider

# Settings for places2processed
tol_d        = 0.75 # Consider a segment developed if it lies closer than tol_d to a developed area
tracktype_p0 = ['grade4','grade5']
tracktype_p1 = ['grade2','grade3']
tracktype_p2 = ['grade1']
surface_p0   = ['ground','grass','dirt','sand','earth','mud']
surface_p1   = ['unpaved','gravel','fine_gravel','wood','compacted','rocks','pebblestone','woodchips','snow','ice','salt']
highway_p1   = ['track','path','footway','bridleway']
types_slow   = ['pedestrian','track','footway','bridleway','steps','corridor','path']
types_heavy  = ['motorway','trunk','primary','secondary','tertiary']

## Loading GPX file

In [6]:
trail = gr_utils.get_gpx(trailname)
trail_coords = gr_mapmatch.trail_to_coords(trail) # Convert the points into a list of [lat, lon] pairs

Loading trail points from <data_input/gr122.gpx>...
Finished loading.


## Matching GPX points to nodes

In [32]:
n_trail = len(trail) # Number of GPX points in the trail
n_batch = int(np.ceil(trail.shape[0]/points_per_batch)) # Number of batches to be run
node_list = pd.DataFrame() # ID of node matched to each GPX point

## --- Loop over all sections of the trail
for b in range(n_batch): # Using batch counter b
# for b in range(3):

    # Define the range of GPX points to process in the current batch
    n1 = b*points_per_batch # First point of this batch
    n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)
    
    print(f'Handling batch #{b}/{n_batch-1} spanning GPX points {n1} through {n2-1}')
    
    trail_section = trail.loc[n1:n2] # Select that range of GPX points
    section_coords = trail_coords[n1:n2] # Convert the points into a list of [lat, lon] pairs

    lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(trail_section,delta) # Calculate the bounding box
    network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM
    new_nodes = gr_mapmatch.match_nodes_vec(network,section_coords) # Calculate corresponding node for each GPX point
    
    # Calculate distance between each GPX point and its corresponding node, in this batch
    dvec = []
    for j in range(len(section_coords)):
        point_id = n1 + j
        point_y = section_coords[j][0]
        point_x = section_coords[j][1]
        node_id = new_nodes[j]
        node = network['points'].loc[node_id]
        node_x = node['x']
        node_y = node['y']
        d2node = gr_mapmatch.get_dist(section_coords[j],[node_y,node_x]) # distance from gpx point to matching node
        newrow = pd.DataFrame({'point_x':point_x, 'point_y':point_y, 'node_id':node_id, 'node_x':node_x, 'node_y':node_y, 'd2node':d2node},index={point_id})
        node_list = pd.concat([node_list,newrow])
    
## --- Save the node_list to a file
filename_nodes = 'cache/' + trailname + '_nodes.csv'
node_list.to_csv(filename_nodes)

Handling batch #0/39 spanning GPX points 0 through 99
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #1/39 spanning GPX points 100 through 199
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #2/39 spanning GPX points 200 through 299
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #3/39 spanning GPX points 300 through 399
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #4/39 spanning GPX points 400 through 499
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #5/39 spanning GPX points 500 through 599
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #6/39 spanning GPX points 600 through 699
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #7/39 spanning GPX points 700 through 799
   Processing street network...
   Handling GPX point 99 of 99...
Handling batch #8/3

In [33]:
node_list

,point_x,point_y,node_id,node_x,node_y,d2node
0,4.05118,51.27809,1649282241,4.051181,51.278079,0.000011
1,4.05111,51.27828,42291968,4.051096,51.278274,0.000015
2,4.05092,51.27827,393406001,4.050765,51.278272,0.000155
3,4.05025,51.27826,1871685302,4.050218,51.278269,0.000034
4,4.04961,51.27826,1871685302,4.050218,51.278269,0.000608
...,...,...,...,...,...,...
3943,3.51948,50.76068,4902407176,3.519843,50.761135,0.000582
3944,3.51984,50.76113,4902407176,3.519843,50.761135,0.000006
3945,3.51998,50.76117,4902407176,3.519843,50.761135,0.000142
3946,3.52009,50.76124,4902407176,3.519843,50.761135,0.000269


## Building "pieces" dataframe

In [8]:
# Loading the node_list
gpx = pd.read_csv(filename_nodes,index_col=0)

In [9]:
pieces = pd.DataFrame()
k = 0 # piece counter
n = gpx.shape[0] # nrows in gpx
j0 = 0
j1 = j0
j2 = j0

while j2<n-1:
    
    node0 = gpx.loc[j0]['node_id']
    j1 = j0 + 1
    node1 = gpx.loc[j1]['node_id']
    
    # increment j1 until we find a different node
    while node1==node0:
        j1 += 1
        node1 = gpx.loc[j1]['node_id']
        
    # increment j2 until we find another node
    j2 = j1
    node2 = gpx.loc[j2]['node_id']
    while node2==node1:
        if j2 == n-1: # we have reached the end of the gpx dataframe
            break
        j2 += 1
        node2 = gpx.loc[j2]['node_id']
        
    if j2 == n-1:
        j3 = j2
    else:
        # find the gpx point between j1 and j2 that is nearest to node1
        temp = gpx.loc[j1:j2-1]
        j3 = temp['d2node'].idxmin()
    
    # pathfinding is to take place between j0 and j3, add to pieces list
    newrow = pd.DataFrame({'node0':node0, 'node1':node1, 'gpx0':j0, 'gpx1':j3},index={k}).astype(int)
    pieces = pd.concat([pieces,newrow])
    k += 1
    
    # move to the next piece by setting j0 = j3
    j0 = j3

In [21]:
## --- Save the pieces list to a file
filename_pieces = 'cache/' + trailname + '_pieces.csv'
with open(filename_pieces, 'w') as file:
    csv_writer = writer(file)
    headers = ['node0','node1','gpx0','gpx1']
    csv_writer.writerow(headers)
    for row in node_list:
        csv_writer.writerow(row)

## Performing pathfinding on each piece

In [10]:
def get_dist(X,Y):
    r = [X[0] - Y[0],
         X[1] - Y[1]]
    return np.sqrt(r[0]*r[0] + r[1]*r[1])

In [11]:
def get_p2seg_distance(P,P0,P1):
    v = [P1[0] - P0[0],
         P1[1] - P0[1]]
    w = [P[0] - P0[0],
         P[1] - P0[1]]
    c1 = w[0]*v[0] + w[1]*v[1]
    if c1<=0:
        return get_dist(P,P0)
    c2 = v[0]*v[0] + v[1]*v[1]
    if c2<=c1:
        return get_dist(P,P1)
    b = c1/c2
    Pb = [P0[0] + b*v[0],
          P0[1] + b*v[1]]
    return get_dist(P,Pb)

In [12]:
def get_coords_from_route(route):
    
    # route is a list of [51.2780788,4.051181,51.2781197,4.0511689,4.625675490697481,5.734,'residential',nan,nan]
    xy = []
    for segment in route:
        xy.append([segment[0],segment[1]])
    segment = route[-1]
    xy.append([segment[2],segment[3]]) # adding the last point
    return xy

In [13]:
def get_path_error(route,trail_coords):
    xy = get_coords_from_route(route)
    err = 0
    for point in trail_coords: # for each point from the gpx
        dmin = 999
        for j in range(len(xy)-1): # find the min distance to the pathfind
            dmin = min(dmin,get_p2seg_distance(point,xy[j],xy[j+1]))
        err += dmin*dmin # save the squared error
    return err

In [14]:
# def pathfind(network, trial_coords, piece):
    
#     j0 = piece['gpx0']
#     j1 = piece['gpx1']
#     points = trail_coords[]

In [15]:
def gpx2points(gpx):
    
    x = gpx['point_x'].values.tolist()
    y = gpx['point_y'].values.tolist()
    points = list(zip(x,y))
    return [[point[1],point[0]] for point in points]

In [16]:
def pathfind(network,node0,node1,points):
    
    # Calculate paths
    route = ox.k_shortest_paths(network['graph'], node0, node1, npaths)
    paths = []
    for path in route:
        paths.append(path)

    # --- Choose the best path
    # Get segment coordinates for each path
    routes = []
    for path in paths:
        segment_list = []
        for j in range(len(path)-1):
            segment_list.extend(gr_mapmatch.get_segments(network, path[j], path[j+1]))
        routes.append(segment_list)

    # Get the error for each path
    err = []
    d = []
    for route in routes:
        err.append(get_path_error(route,points))
        d_osm = 0
        for segment in route:
            d_osm += segment[5] # add d_osm column
        d.append(d_osm)
    dmin = min(d)
    weights = []
    if row['gpx1'] - row['gpx0'] < 4:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(d[j])
    else:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(err[j]*np.exp(d[j]/dmin))

    # Choose the best route
    imin = weights.index(min(weights))
    return routes[imin]

In [19]:
def print_overwrite(text):
    
    print(text, end='\r', flush=True)

In [20]:
npaths = 5 # number of routes to find
n1 = 0 # First point of this batch
n2 = min(n1 + points_per_batch, n_trail) # Last point of this batch (clipped)
lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(trail.loc[n1:n2],delta) # Calculate the bounding box
network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM

total_route = []

for idx, row in pieces.iterrows():
    
    print_overwrite(f"\rHandling piece {idx} of {pieces.shape[0]}, spanning GPX points {row['gpx0']} through {row['gpx1']}")

    # --- Grab GPX points to be used in error calculation
    gpx_select = gpx.loc[row['gpx0']:row['gpx1']] 
    x = gpx_select['point_x'].values.tolist()
    y = gpx_select['point_y'].values.tolist()
    temp = list(zip(x,y))
    points = [[item[1],item[0]] for item in temp]
    
    # --- Check if we need to download the next network
    if row['gpx0']>n2:
        n1 += points_per_batch
        n2 += points_per_batch
        print('')
        lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(trail.loc[n1:n2],delta) # Calculate the bounding box
        network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM

    # --- Calculate paths
    found = False
    ntry = 1
    while not found: # in case the node is not in the network, reload it with bigger delta
        try:
            route = ox.k_shortest_paths(network['graph'], row['node0'], row['node1'], npaths)
            paths = []
            for path in route:
                paths.append(path)
        except:
            print('')
            print('   Node not found in network, downloading larger network...')
            lat_min, lat_max, lon_min, lon_max = gr_mapmatch.get_bbox(trail.loc[n1:n2],ntry*delta) # Calculate the bounding box
            network = gr_mapmatch.get_osm_network(lat_min, lat_max, lon_min, lon_max) # Download the street network from OSM
            ntry += 1
        else:
            found = True
        
    # --- Choose the best path
    
    # Get segment coordinates for each path
    routes = []
    for path in paths:
        segment_list = []
        for j in range(len(path)-1):
            segment_list.extend(gr_mapmatch.get_segments(network, path[j], path[j+1]))
        routes.append(segment_list)

    # Get the error for each path
    err = []
    d = []
    for route in routes:
        err.append(get_path_error(route,points))
        d_osm = 0
        for segment in route:
            d_osm += segment[5] # add d_osm column
        d.append(d_osm)
    dmin = min(d)
    weights = []
    if row['gpx1'] - row['gpx0'] < 4:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(d[j])
    else:
        for j in range(len(err)): # Weight factor takes into account error and length
            weights.append(err[j]*np.exp(d[j]/dmin))
        
    # Choose the best route
    imin = weights.index(min(weights))
    best_route = routes[imin]
    total_route.extend(best_route)

   Processing street network...
Handling piece 38 of 796, spanning GPX points 101 through 102
   Processing street network...
Handling piece 53 of 796, spanning GPX points 217 through 229
   Processing street network...
Handling piece 69 of 796, spanning GPX points 301 through 307
   Processing street network...
Handling piece 94 of 796, spanning GPX points 402 through 404
   Processing street network...
Handling piece 117 of 796, spanning GPX points 504 through 505
   Processing street network...
Handling piece 133 of 796, spanning GPX points 606 through 609
   Processing street network...
Handling piece 157 of 796, spanning GPX points 773 through 801
   Processing street network...
Handling piece 158 of 796, spanning GPX points 801 through 804
   Processing street network...
Handling piece 192 of 796, spanning GPX points 908 through 913
   Processing street network...
Handling piece 218 of 796, spanning GPX points 1006 through 1009
   Processing street network...
Handling piece 244 o

In [23]:
# Placing total_route into a dataframe
segments = pd.DataFrame(total_route, columns=['x0','y0','x1','y1','d_cart','d_osm','highway','surface','tracktype'])
# And save it
filename_segments = 'cache/' + trailname + '_segments.csv'
segments.to_csv(filename_segments)

## Plotting the matched route

In [24]:
matchedcoords = gr_plot.get_fullcoords_from_frame(segments)

# Map setup
mid = int(len(trail_coords)/2)
chart = folium.Map(location=trail_coords[mid], zoom_start=10, tiles="OpenStreetMap")

# Draw the GPX route
newline = folium.PolyLine(locations=trail_coords, weight=2, color='red')
newline.add_to(chart)

# Draw the matched route
newline = folium.PolyLine(locations=matchedcoords, weight=2, color='black')
newline.add_to(chart)

# Render the map
filepath = f"cache/temp11.html"
chart.save(filepath)
IFrame(filepath, width=1000, height=500)